In [1]:
library(tidyverse)

Warning message:
"package 'ggplot2' was built under R version 4.3.3"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Creating grade-like data
This simulated file contains grade data like one could get from the eLC gradebook. Each student is given a single row with each exam as a separate column.

In [177]:
data1 = data.frame(id= 1:100)
data2 = data.frame(id = 1:100)

set.seed(42)
data1 <- data1 %>%
        mutate(group="group1",
            exam1 = round(rnorm(100, 85, 5),0),
              exam2 = exam1 + round(rnorm(100, 5, 1),0),
               exam3 = round((exam1+ exam2)/2 + rnorm(100, 5, 1),0),
              across(c(exam1, exam2, exam3), ~ if_else(. >100, 100, .))

              )

set.seed(1785)
data2 <- data2 %>%
        mutate(group = "group2",
            exam1 = round(rnorm(100, 82, 4),0),
              exam2 = exam1 + round(rnorm(100, 4, 1),0),
               exam3 = round((exam1+ exam2)/2 + rnorm(100, 4, 1),0),
              across(c(exam1, exam2, exam3), ~ if_else(. >100, 100, .))
              )

data <- rbind.data.frame(data1,data2)

set.seed(1)
data <- data %>%
            slice_sample(n=nrow(data)) %>%
            mutate(id = 1000 + 1:nrow(data))

In [179]:
head(data, 10)

,id,group,exam1,exam2,exam3
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,1001,group1,90,95,97
2,1002,group2,77,80,81
3,1003,group2,80,84,86
4,1004,group2,89,93,95
5,1005,group1,89,94,98
6,1006,group1,84,88,91
7,1007,group2,83,87,89
8,1008,group1,87,92,93
9,1009,group1,79,84,87


In [181]:
write.csv(data, file="exam_data.csv", row.names=FALSE)

## With missing data

In [225]:
set.seed(123)

missing_rows <- sample(1:200, 5, replace=FALSE)
missing_columns1 <- sample(1:200, 5, replace=FALSE)
missing_columns2<- sample(1:200, 5, replace=FALSE)
missing_columns3 <- sample(1:200, 5, replace=FALSE)

data1m<-data %>% 
            mutate(
                    Row_number = row_number(),
                    across(c(exam1, exam2, exam3), ~if_else(Row_number %in% missing_rows, NA_real_, .)),
                    exam1= if_else(Row_number %in% missing_columns1, NA_real_, exam1),
                    exam2= if_else(Row_number %in% missing_columns2, NA_real_, exam2),
                    exam3= if_else(Row_number %in% missing_columns3, NA_real_, exam3)
                  ) %>%
            select(-Row_number)
            

In [227]:
write.csv(data1m, file="exam_data_missing.csv", row.names=FALSE)

# Test question data

In [142]:
data3 = data.frame(id= 100:199)
data4 = data.frame(id = 200:299)

In [170]:
set.seed(42)

data3 <- data3 %>%
                mutate(section = "morning",
                       Q1 = sample(c(0,1), size=100, replace=TRUE),
                       Q2 = sample(c(0,1), size=100, replace=TRUE),
                       Q3 = sample(c(0,1), size=100, replace=TRUE, prob = c(0.4,.6)),
                       Q4 = sample(c(0,1), size=100, replace=TRUE, prob = c(0.8,.2)),
        
                       )

write.csv(data3, file="morning_exam.csv", row.names=FALSE)

In [172]:
set.seed(1785)
data4 <- data4 %>%
                mutate(section= "afternoon",
                       Q1 = sample(c(0,1), size=100, replace=TRUE),
                       Q2 = sample(c(0,1), size=100, replace=TRUE, prob = c(0.36,.64)),
                       Q3 = sample(c("A","B", "C", "D"), size=100, replace=TRUE, prob = c(0.1, 0.5, 0.25, 0.15)),
                       Q4 = sample(c(0,1), size=100, replace=TRUE, prob = c(0.82,.18)),
        
                       )

write.csv(data4, file="afternoon_exam.csv", row.names=FALSE)

In [174]:
set.seed(1776)
demographics <- data.frame(id = 100:299) %>%
                    mutate(group1 = sample(c(0, 1), size = 200, replace=TRUE, prob=c(0.75, 0.25)),
                           group2 = sample(c(0, 1), size = 200, replace=TRUE)

                           )

write.csv(demographics, file="class_demographics.csv", row.names=FALSE)

# Exam grades part

In [230]:
exam_data <- read.csv("exam_data.csv")

In [236]:
exam_data %>%
    summarize(exam1_mean = mean(exam1),
              exam2_mean = mean(exam2),
              exam3_mean = mean(exam3)
              )

exam1_mean,exam2_mean,exam3_mean
<dbl>,<dbl>,<dbl>
83.62,88.035,90.27


In [238]:
exam_data %>%
    rowwise() %>%
    mutate(avg_grade = mean(c(exam1, exam2, exam3)))

id,group,exam1,exam2,exam3,avg_grade
<int>,<chr>,<int>,<int>,<int>,<dbl>
1001,group1,90,95,97,94.00000
1002,group2,77,80,81,79.33333
1003,group2,80,84,86,83.33333
1004,group2,89,93,95,92.33333
1005,group1,89,94,98,93.66667
1006,group1,84,88,91,87.66667
1007,group2,83,87,89,86.33333
1008,group1,87,92,93,90.66667
1009,group1,79,84,87,83.33333


In [240]:
exam_data %>%
    group_by(group) %>%
    summarize(exam1_mean = mean(exam1),
              exam2_mean = mean(exam2),
              exam3_mean = mean(exam3)
              )

group,exam1_mean,exam2_mean,exam3_mean
<chr>,<dbl>,<dbl>,<dbl>
group1,85.15,90.09,92.54
group2,82.09,85.98,88.00


In [242]:
exam_data %>%
    arrange(group)

id,group,exam1,exam2,exam3
<int>,<chr>,<int>,<int>,<int>
1001,group1,90,95,97
1005,group1,89,94,98
1006,group1,84,88,91
1008,group1,87,92,93
1009,group1,79,84,87
1010,group1,83,87,90
1013,group1,80,87,89
1014,group1,93,98,100
1015,group1,88,91,95


In [246]:
exam_data %>%
    rowwise() %>%
    mutate(avg_grade = mean(c(exam1, exam2, exam3))) %>%
    filter(group == "group1") %>%
    select(avg_grade)

avg_grade
<dbl>
94.00000
93.66667
87.66667
90.66667
83.33333
86.66667
85.33333
97.00000
91.33333


In [248]:
exam_data %>%
    rowwise() %>%
    mutate(avg_grade = mean(c(exam1, exam2, exam3))) %>%
    filter(group == "group1") %>%
    pull(avg_grade)

[1] 94.00000 93.66667 87.66667 90.66667 83.33333 86.66667 85.33333 97.00000
  [9] 91.33333 85.66667 85.66667 86.66667 93.00000 93.66667 88.33333 93.66667
 [17] 87.00000 95.66667 85.66667 89.33333 89.00000 97.66667 77.33333 94.33333
 [25] 81.66667 79.66667 86.66667 82.33333 88.00000 90.33333 96.00000 91.66667
 [33] 87.66667 89.00000 92.33333 95.66667 96.00000 85.66667 90.66667 92.00000
 [41] 89.00000 96.00000 73.33333 87.00000 87.33333 88.66667 92.66667 83.00000
 [49] 97.33333 91.66667 92.66667 83.00000 91.33333 79.00000 87.33333 92.33333
 [57] 91.00000 84.33333 88.00000 89.33333 89.66667 91.33333 87.33333 96.00000
 [65] 92.33333 76.66667 79.66667 86.66667 82.66667 90.00000 88.33333 95.66667
 [73] 97.00000 93.33333 92.66667 90.00000 98.33333 84.66667 85.66667 91.66667
 [81] 93.33333 90.33333 98.66667 85.66667 87.66667 96.66667 92.00000 91.00000
 [89] 89.66667 88.00000 90.33333 85.66667 84.33333 92.33333 86.66667 78.00000
 [97] 88.66667 94.00000 89.00000 92.33333

# Question data

In [265]:
a_data <- read.csv("afternoon_exam.csv")
m_data <- read.csv("morning_exam.csv")

In [269]:
head(a_data)

,id,section,Q1,Q2,Q3,Q4
,<int>,<chr>,<int>,<int>,<chr>,<int>
1,200,afternoon,0,0,B,0
2,201,afternoon,1,0,C,0
3,202,afternoon,1,1,B,0
4,203,afternoon,0,0,B,0
5,204,afternoon,0,1,D,0
6,205,afternoon,1,0,B,0


In [281]:
a_data %>%
    mutate(Q3 = recode(Q3, "A"=0, "B"=1, "C"=0, "D"=0))

a_data %>%
    mutate(Q3 = if_else(Q3=="B", 1, 0))

a_data %>%
    mutate(Q3= case_when(
        Q3=="A" ~ 0,
        Q3=="B" ~ 1,
        Q3=="C" ~ 0,
        Q3=="D" ~ 0
        )
           )

id,section,Q1,Q2,Q3,Q4
<int>,<chr>,<int>,<int>,<dbl>,<int>
200,afternoon,0,0,1,0
201,afternoon,1,0,0,0
202,afternoon,1,1,1,0
203,afternoon,0,0,1,0
204,afternoon,0,1,0,0
205,afternoon,1,0,1,0
206,afternoon,1,1,0,0
207,afternoon,0,1,1,0
208,afternoon,0,0,1,0


id,section,Q1,Q2,Q3,Q4
<int>,<chr>,<int>,<int>,<dbl>,<int>
200,afternoon,0,0,1,0
201,afternoon,1,0,0,0
202,afternoon,1,1,1,0
203,afternoon,0,0,1,0
204,afternoon,0,1,0,0
205,afternoon,1,0,1,0
206,afternoon,1,1,0,0
207,afternoon,0,1,1,0
208,afternoon,0,0,1,0


id,section,Q1,Q2,Q3,Q4
<int>,<chr>,<int>,<int>,<dbl>,<int>
200,afternoon,0,0,1,0
201,afternoon,1,0,0,0
202,afternoon,1,1,1,0
203,afternoon,0,0,1,0
204,afternoon,0,1,0,0
205,afternoon,1,0,1,0
206,afternoon,1,1,0,0
207,afternoon,0,1,1,0
208,afternoon,0,0,1,0
